In [48]:
import os
import pandas as pd
import numpy as np

In [49]:
df = pd.read_csv("고객여정.csv", engine = "python")
df.head()

,고객ID,날짜,행동
0,C_1,2020-01-03,A7
1,C_1,2020-01-08,A6
2,C_1,2020-02-06,A5
3,C_1,2020-02-17,A5
4,C_1,2020-02-18,A7


In [50]:
# 시계열 데이터라 볼 수 있음 -> 순서 중요
df.sort_values(by=['고객ID', '날짜'], inplace=True)

In [52]:
action_set = df['행동'].unique()

In [53]:
churn_ID = df.loc[df['행동']=='이탈', '고객ID'].unique()
not_churn_ID = df.loc[~df['고객ID'].isin(churn_ID), '고객ID'].unique()

In [54]:
churn_df = df.loc[df['고객ID'].isin(churn_ID)]
not_churn_df = df.loc[df['고객ID'].isin(not_churn_ID)]

In [56]:
churn_action_sequence = churn_df.groupby(['고객ID'])['행동'].apply(np.array)
not_churn_action_sequence = not_churn_df.groupby(['고객ID'])['행동'].apply(np.array)

In [57]:
from itertools import product # 가능한 모든 조합을 출력해줌

def contain_pattern(record, pattern, L):
    # 특정 record하에 특정 pattern이 발생했는지
    # L:윈도우 크기
    output = False
    if set(record) & set(pattern) != set(pattern): # pattern에 포함된 모든 아이템 집합이 record에 포함된 아이템 집합에 속하지 않으면
        # 해석: record와 pattern의 교집합이 pattern과 같지 않다 => pattern이 record에 속하지 않는다
        return False
    
    else:
        # 패턴에 속한 개별 아이템에 대한 위치를 미리 구하기
        # pattern에 있는 item을 돌면서 record의 어느 위치에 있는지 확인
        # [0]: np.where의 output이 array임. 그중 0번째를 가져오겠다는 것
        pattern_index_list = [np.where(record == item)[0] for item in pattern]
        
        ## 가능한 모든 조합에서 위치 간 거리가 L이하면 True를 반환
        # Ex) record = [A, B, C, A, C, C], pattern = [A, B], L = 1
        # A의 위치: [0, 3], B의 위치: [1]
        # 가능한 모든 조합(product): [0, 1], [3, 1](얘는 A가 3번째 있고 B가 첫번째 있으므로 L에 상관없이 우리가 원하는 패턴이 아냐 =>그래서 밑에 distance를 구함)
        # 가능한 모든 조합의 거리 차이: [1-0, 1-3] 중에 0번째 요소는 만족하므로 True
        
        for pattern_index in product(*pattern_index_list):
            # product는 모든 조합을 찾음
            distance = np.array(pattern_index)[1:] - np.array(pattern_index)[:-1]
            # [1:]는 뒤에 있는 요소, [:-1]는 앞에 있는 요소
            if sum((distance <= L) & (distance > 0)) == (len(pattern_index) - 1): # -1은 맨 앞에 있는 것은 거리상 무시됐기때문에
                output = True
                break
                
        return output

In [58]:
def find_maximum_frequent_sequence_item(item_set, sequence_data, min_support=0.01, L=1):
    queue = []
    maximum_frequent_sequence_item = []
    
    # 유니크한 아이템 집합에 대해, min_support가 넘는 아이템들만 queue에 추가
    for item in item_set:
        occurence = sequence_data.apply(contain_pattern, pattern=[item], L=L).sum()
        if occurence / len(sequence_data) >= min_support:
            # 등장비율
            queue.append([item])
            
    while queue:
        current_pattern = queue.pop()  # 맨 마지막 값 빼기
        check_maximum_frequent = True # 모든 자식 집합이 min_support를 넘기지 않으면 True를 유지
        for item in item_set:
            occurence = sequence_data.apply(contain_pattern, pattern=current_pattern + [item], L=L).sum()
            if occurence / len(sequence_data) >= min_support: # min_support를 넘는 패턴을 queue에 추가
                check_maximum_frequent = False
                queue.append(current_pattern + [item])
                
        if check_maximum_frequent and len(current_pattern) > 1:
            maximum_frequent_sequence_item.append(current_pattern)
            
    return maximum_frequent_sequence_item

In [64]:
def generate_association_rules(maximum_frequent_sequence_item, sequence_data, min_support = 0.01, min_confidence = 0.5, L = 1):
    # 결과 초기화
    result = []
        
    for sequence_item in maximum_frequent_sequence_item:
        # A -> B에서 A, B를 모두 포함하는 가짓 수 co_occurence 계산
        co_occurence = sequence_data.apply(contain_pattern, pattern = sequence_item, L = L).sum()
        support = co_occurence / len(sequence_data)
        if co_occurence > min_support:
            for i in range(len(sequence_item)-1, 0, -1): # 한 아이템 집합에 대해, 부모의 크기를 1씩 줄여나가는 방식으로 부모와 자식 설정
                antecedent = sequence_item[:i]
                consequent = sequence_item[i:]
                antecedent_occurence = sequence_data.apply(contain_pattern, pattern = antecedent, L = L).sum()
               
                confidence = co_occurence / antecedent_occurence
                if confidence > min_confidence:
                    result.append(sequence_item)            
    
    return result

In [65]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(action_set, not_churn_action_sequence, min_support = 0.1, L = 1)
not_churn_rules = generate_association_rules(maximum_frequent_sequence_item, not_churn_action_sequence, min_support=0.1,  min_confidence = 0.1, L = 1)

In [66]:
maximum_frequent_sequence_item = find_maximum_frequent_sequence_item(action_set, churn_action_sequence, min_support = 0.1, L = 1)
churn_rules = generate_association_rules(maximum_frequent_sequence_item, churn_action_sequence, min_support = 0.1, min_confidence = 0.1, L = 1)

In [67]:
churn_rules[:5]

[['A2', 'A2'],
 ['A2', 'A4'],
 ['A2', 'A5'],
 ['A2', 'A6', 'A7'],
 ['A2', 'A6', 'A7']]

In [68]:
not_churn_rules[:5]

[['A2', 'A3'], ['A2', 'A1'], ['A2', 'A5'], ['A2', 'A6'], ['A3', 'A2']]

In [70]:
result = []

for rule in churn_rules + not_churn_rules:
    if '이탈' in rule:
        rule = rule[:-1] # 이탈 부분 빼기위해
        
    churn_support = churn_action_sequence.apply(contain_pattern, pattern=rule, L=1).sum()/len(churn_action_sequence)
    not_churn_support = not_churn_action_sequence.apply(contain_pattern, pattern=rule, L=1).sum()/len(not_churn_action_sequence)
    
    result.append(['-'.join(rule), churn_support, not_churn_support])
    
pd.DataFrame(result, columns=['Rule', 'churn_support', 'not_churn_support'])

# 좀 더 나아가 둘의 support 차이가 통계적으로 유의미한지 확인하면 좋겠지

,Rule,churn_support,not_churn_support
0,A2-A2,0.210000,0.055714
1,A2-A4,0.150000,0.060000
2,A2-A5,0.220000,0.295714
3,A2-A6-A7,0.100000,0.041429
4,A2-A6-A7,0.100000,0.041429
...,...,...,...
111,A7-A4,0.406667,0.195714
112,A7-A1,0.013333,0.277143
113,A7-A5,0.216667,0.164286
114,A7-A6,0.433333,0.264286
